In [2]:
from model import DESModel, HydroDynamicModel
import openclsim.plot as plot
import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr

In [3]:
m = DESModel()
m.resources()
m.processes(limit_expr=lambda Hm0: Hm0 > 0.5)
m.start_simulation()

In [4]:
plot.get_log_dataframe(m.process)


,Activity,Timestamp,ActivityState,type,ref
0,e170d3c6-6fc8-48db-be60-3948d9d45646,2018-01-01 00:00:00,WAIT_START,plugin,waiting on weather
1,e170d3c6-6fc8-48db-be60-3948d9d45646,2018-01-11 05:00:00,WAIT_STOP,plugin,waiting on weather
2,e170d3c6-6fc8-48db-be60-3948d9d45646,2018-01-11 05:00:00,START,NaN,NaN
3,e170d3c6-6fc8-48db-be60-3948d9d45646,2018-01-11 06:00:00,STOP,NaN,NaN


In [5]:
m.project_length()

Timedelta('10 days 06:00:00')

In [6]:
df = m.weather_resource.conditions_database.set_index("Timestamp").copy()
df = df[["Hm0", "MWD" ,"Tp"]]

In [7]:
# Pandas 2D RAOs to 3D Xarray.
rao = pd.read_csv("./data/preprocessed/MPI_Adventure_RAO.csv")
datasets = np.array_split(rao.set_index(["RAOPeriodOrFreq", "dir"]), 12, axis=1)
datasets = [data.pivot_table(index="RAOPeriodOrFreq", columns="dir") for data in datasets]

rao_da = xr.DataArray(
    datasets, 
    dims=["DOF", "Freq", "Dir"], coords={
        "DOF": ["SurgeAmp", "SurgePhase", "SwayAmp", "SwayPhase", "HeaveAmp", "HeavePhase", "RollAmp", "RollPhase", "PitchAmp", "PitchPhase", "YawAmp", "YawPhase"],
        "Freq": datasets[0].index.values,
        "Dir": (datasets[0].columns.droplevel(0).values % 360)
    }
).sortby("Dir")

In [8]:
# Setup wave heading.
heading = 270

# Define wave parameters.
H_swell, T_swell, D_swell = (.5, 9, (180 + heading - 315) % 360)
H_windsea, T_windsea, D_windsea = (2, 5, (180 + heading - 225) % 360)

# Get uncoupled responses.
swell_response = rao_da.sel(dict(Freq=T_swell, Dir=D_swell), method="nearest") * H_swell 
windsea_response = rao_da.sel(dict(Freq=T_windsea, Dir=D_windsea), method="nearest") * H_windsea

In [9]:
(swell_response + windsea_response).to_pandas()

DOF
SurgeAmp        0.0865
SurgePhase   -193.7005
SwayAmp         0.0725
SwayPhase    -131.7545
HeaveAmp        0.1595
HeavePhase    186.5610
RollAmp         0.1780
RollPhase     140.2330
PitchAmp        0.6160
PitchPhase    -57.6650
YawAmp          0.3430
YawPhase     -167.5500
dtype: float64

In [27]:
rao_da.sel(dict(Freq=df["Tp"], Dir=D_swell), method="nearest").to_pandas()

Freq,15.830,15.830,15.830,15.830,15.830,15.830,15.830,15.830,15.830,15.830,...,5.380,5.380,5.535,9.049,8.323,8.323,8.323,7.759,7.759,7.759
DOF,,,,,,,,,,,,,,,,,,,,,
SurgeAmp,0.958,0.958,0.958,0.958,0.958,0.958,0.958,0.958,0.958,0.958,...,0.027,0.027,0.032,0.093,0.019,0.019,0.019,0.032,0.032,0.032
SurgePhase,-89.619,-89.619,-89.619,-89.619,-89.619,-89.619,-89.619,-89.619,-89.619,-89.619,...,-61.007,-61.007,-38.175,-81.325,-50.646,-50.646,-50.646,64.178,64.178,64.178
SwayAmp,0.777,0.777,0.777,0.777,0.777,0.777,0.777,0.777,0.777,0.777,...,0.012,0.012,0.005,0.093,0.028,0.028,0.028,0.038,0.038,0.038
SwayPhase,90.014,90.014,90.014,90.014,90.014,90.014,90.014,90.014,90.014,90.014,...,50.230,50.230,162.561,87.727,125.977,125.977,125.977,-141.310,-141.310,-141.310
HeaveAmp,0.702,0.702,0.702,0.702,0.702,0.702,0.702,0.702,0.702,0.702,...,0.027,0.027,0.027,0.263,0.182,0.182,0.182,0.138,0.138,0.138
HeavePhase,-1.431,-1.431,-1.431,-1.431,-1.431,-1.431,-1.431,-1.431,-1.431,-1.431,...,113.415,113.415,140.391,-24.946,-40.228,-40.228,-40.228,-64.232,-64.232,-64.232
RollAmp,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.999,...,0.080,0.080,0.080,0.160,0.293,0.293,0.293,0.285,0.285,0.285
RollPhase,109.135,109.135,109.135,109.135,109.135,109.135,109.135,109.135,109.135,109.135,...,-103.529,-103.529,-102.187,57.154,83.991,83.991,83.991,91.181,91.181,91.181
PitchAmp,1.124,1.124,1.124,1.124,1.124,1.124,1.124,1.124,1.124,1.124,...,0.067,0.067,0.092,1.036,0.804,0.804,0.804,0.553,0.553,0.553
